In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

# NOS

In [ ]:
nos_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/dutch-news-articles.csv")
nos_sample = nos_dataset.sample(4000, replace=False, random_state=1)
nos_sample.to_csv("nos.csv")

# Niburu

In [ ]:
niburu_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/niburu.csv")
print(niburu_dataset.shape)
#Remove whitespace
niburu_dataset["title"] = niburu_dataset["title"].str.strip()
niburu_dataset["article"] = niburu_dataset["article"].str.replace("\n"," ")
niburu_dataset["article"] = niburu_dataset["article"].str.replace("  ", " ")
niburu_dataset["article"] = niburu_dataset["article"].str.strip()
niburu_dataset["date"] = niburu_dataset["date"].str.strip()
#Drop rows without article
niburu_dataset = niburu_dataset[niburu_dataset["article"] != "None"]
niburu_dataset = niburu_dataset.drop_duplicates()
niburu_dataset = niburu_dataset[["url","title","article","date"]]
niburu_dataset["publisher"] = "niburu"
niburu_dataset["trust"] = "untrusted"
print(niburu_dataset.shape)
niburu_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/niburu.csv", index=False)

(4459, 4)
(4459, 6)


# DpgMedia2019

In [ ]:
dpg_dataset = pd.read_json("/content/drive/MyDrive/Master Thesis/dpgMedia2019-articles-bypublisher.jsonl", lines=True)
dpg_labels = pd.read_json("/content/drive/MyDrive/Master Thesis/dpgMedia2019-labels-bypublisher.jsonl", lines=True)
combined = dpg_dataset.merge(dpg_labels, how ="inner", on="id")
# Remove title from text so that only the article text remains.
combined["text"] = combined.apply(lambda x: x["text"].replace(x["title"], ""), axis=1)
# Drop the partisan column
combined = combined.drop("partisan", axis=1)

# Split the dataset in separate ones for volkskrant, trouw, ad, parool and regional.
ad = combined[combined["publisher"] == "ad"]
volkskrant = combined[combined["publisher"] == "vk"]
trouw = combined[combined["publisher"] == "trouw"]
parool = combined[combined["publisher"] == "parool"]
regional = combined[(combined["publisher"] == "eindhovensdagblad") | (combined["publisher"] == "pzc") |
                    (combined["publisher"] == "destentor") | (combined["publisher"] == "degelderlander") |
                    (combined["publisher"] == "brabantsdagblad") | (combined["publisher"] == "destem") |
                    (combined["publisher"] == "tubantia")]

# Randomly sample instances from the datasets.

ad_sample = ad.sample(2500, replace=False, random_state=1)
volkskrant_sample = volkskrant.sample(2500, replace=False, random_state=1)
trouw_sample = trouw.sample(5000, replace=False, random_state=1)
parool_sample = parool.sample(2500, replace=False, random_state=1)
regional_sample = regional.sample(2500, replace=False, random_state=1)

print(regional_sample["publisher"].value_counts())

# Generate csv files for the samples

ad_sample.to_csv("ad.csv", index=False)
volkskrant_sample.to_csv("volkskrant.csv", index=False)
trouw_sample.to_csv("trouw.csv", index=False)
parool_sample.to_csv("parool.csv", index=False)
regional_sample.to_csv("regional.csv", index=False)

In [ ]:
nu_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nu.csv")
print(nu_dataset.shape)
nu_dataset = nu_dataset.drop_duplicates()
nu_dataset = nu_dataset[nu_dataset["article"] != "None"]
print(nu_dataset.shape)
nu_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Datasets/nu.csv", index=False)

(3013, 4)
(2124, 4)


In [ ]:
ad_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/ad.csv")
print(ad_dataset.info())
# Drop columns that we do not necessary
ad_dataset = ad_dataset.drop(["id","mainSection"], axis=1)
# Rename text and published_at to article and date
ad_dataset = ad_dataset.rename(columns={"text": "article", "published_at": "date"})
# Convert date column to datetime type
ad_dataset["date"] = pd.to_datetime(ad_dataset["date"])
# Check for duplicates, to be certain
ad_dataset = ad_dataset.drop_duplicates()
# There is one instance without title, let's remove it
ad_dataset = ad_dataset[ad_dataset["title"].notna()]
# Reorder columns
ad_dataset = ad_dataset[["url","title","article","date","publisher"]]
# Add trust column
ad_dataset["trust"] = "trusted"
print(ad_dataset.info())
ad_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/ad.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            2500 non-null   int64 
 1   title         2499 non-null   object
 2   text          2500 non-null   object
 3   mainSection   2500 non-null   object
 4   published_at  2500 non-null   object
 5   publisher     2500 non-null   object
 6   url           2500 non-null   object
dtypes: int64(1), object(6)
memory usage: 136.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2499 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        2499 non-null   object        
 1   title      2499 non-null   object        
 2   article    2499 non-null   object        
 3   date       2499 non-null   datetime64[ns]
 4   publisher  2499 non-null   object        
 5   trust      2499 non

In [ ]:
dlm_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/dlm.csv")
print(dlm_dataset.info())
# First, lets drop instances without articles
dlm_dataset = dlm_dataset[dlm_dataset["article"].notna()]
print(dlm_dataset.info())
# Check for duplicates
dlm_dataset = dlm_dataset.drop_duplicates()
print(dlm_dataset.info())
#Remove repeated author name
dlm_dataset["article"] = dlm_dataset["article"].str.replace("\s*Rob Vellekoop, [0-9][0-9] [a-z]* 20[0-9][0-9]", "")
# Create publisher and trust columns
dlm_dataset["publisher"] = "dlmplus"
dlm_dataset["trust"] = "untrusted"
dlm_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/dlm.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890 entries, 0 to 889
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      890 non-null    object
 1   title    890 non-null    object
 2   article  887 non-null    object
 3   date     890 non-null    object
dtypes: object(4)
memory usage: 27.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 889
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      887 non-null    object
 1   title    887 non-null    object
 2   article  887 non-null    object
 3   date     887 non-null    object
dtypes: object(4)
memory usage: 34.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 0 to 889
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      887 non-null    object
 1   title    887 non-null    object
 2  

In [ ]:
n4n_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/ninefornews.csv")
print(n4n_dataset.info())
# Remove instances which are None
n4n_dataset = n4n_dataset[n4n_dataset["article"] != "None"]
# Check for duplicates
n4n_dataset = n4n_dataset.drop_duplicates()
# Rename time column to date
n4n_dataset = n4n_dataset.rename(columns={"time":"date"})
# Convert date column to datetime
n4n_dataset["date"] = pd.to_datetime(n4n_dataset["date"])
# Add publisher and trust columns
n4n_dataset["publisher"] = "ninefornews"
n4n_dataset["trust"] = "untrusted"
n4n_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/ninefornews.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3076 entries, 0 to 3075
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      3076 non-null   object
 1   title    3076 non-null   object
 2   article  3076 non-null   object
 3   time     3076 non-null   object
dtypes: object(4)
memory usage: 96.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3055 entries, 0 to 3075
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      3055 non-null   object
 1   title    3055 non-null   object
 2   article  3055 non-null   object
 3   time     3055 non-null   object
dtypes: object(4)
memory usage: 119.3+ KB
None


In [ ]:
np_bin_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_bin.csv")
np_bui_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_bui.csv")
np_cul_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_cul.csv")
np_econ_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_econ.csv")
np_lif_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_lif.csv")
np_regio_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_regio.csv")
np_show_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_show.csv")
np_sport_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nieuwspaal_sport.csv")

frames = [np_bin_dataset, np_bui_dataset, np_cul_dataset, np_econ_dataset, np_lif_dataset, np_regio_dataset,
          np_show_dataset, np_sport_dataset]
np_dataset = pd.concat(frames)
print(np_dataset.info())
# Remove articles containing tweets
np_dataset = np_dataset[np_dataset["article"].str.contains("\(@[A-Za-z]*") == False]
# Check for duplicates
np_dataset = np_dataset.drop_duplicates()
# Remove instances without articles
np_dataset = np_dataset[np_dataset["article"] != "None"]
# Convert date column to datetime
np_dataset["date"] = pd.to_datetime(np_dataset["date"])
# Add publisher and trust columns
np_dataset["publisher"] = "nieuwspaal"
np_dataset["trust"] = "untrusted"
print(np_dataset.info())
np_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/nieuwspaal.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1087 entries, 0 to 139
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      1087 non-null   object
 1   title    1087 non-null   object
 2   article  1087 non-null   object
 3   date     1087 non-null   object
dtypes: object(4)
memory usage: 42.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1082 entries, 0 to 139
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        1082 non-null   object        
 1   title      1082 non-null   object        
 2   article    1082 non-null   object        
 3   date       1082 non-null   datetime64[ns]
 4   publisher  1082 non-null   object        
 5   trust      1082 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 59.2+ KB
None


In [ ]:
sp_bin_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/speld_bin.csv")
sp_bui_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/speld_bui.csv")
sp_cul_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/speld_cul.csv")
sp_wet_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/speld_wet.csv")
sp_lif_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/speld_lif.csv")
sp_pol_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/speld_pol.csv")

frames = [sp_bin_dataset,sp_bui_dataset,sp_cul_dataset,sp_wet_dataset,sp_lif_dataset,sp_pol_dataset]
sp_dataset = pd.concat(frames)
print(sp_dataset.info())
# Strip newlines
sp_dataset["title"] = sp_dataset["title"].str.strip()
sp_dataset["time"] = sp_dataset["time"].str.strip()
# Rename time column to date
sp_dataset = sp_dataset.rename(columns={"time": "date"})
# Strip off "door .... part"
sp_dataset["date"] = sp_dataset["date"].str.split().str[0:3].str.join("-")
# Remove instances with weird code in it
sp_dataset = sp_dataset[sp_dataset["article"].str.contains("if\(window.") == False]
# Remove duplicates
sp_dataset = sp_dataset.drop_duplicates()
# Remove instances without articles
sp_dataset = sp_dataset[sp_dataset["article"] != "None"]
# Add publisher and trust column
sp_dataset["publisher"] = "speld"
sp_dataset["trust"] = "untrusted"
sp_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/speld.csv", index=False)

In [ ]:
nos_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nos.csv")
nos_dataset.info()
# Drop columns we do not need
nos_dataset = nos_dataset.drop(nos_dataset.columns[[0,4]], axis=1)
# Rename columns
nos_dataset = nos_dataset.rename(columns={"datetime":"date", "content":"article"})
# Re-order columns
nos_dataset = nos_dataset[["url","title","article","date"]]
# Convert date column to datetime
nos_dataset["date"] = pd.to_datetime(nos_dataset["date"])
# Retain only the date
nos_dataset["date"] = nos_dataset["date"].dt.date
# Check for duplicates
nos_dataset = nos_dataset.drop_duplicates()
# Add publisher and trust column
nos_dataset["publisher"] = "nos"
nos_dataset["trust"] = "trusted"
nos_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/nos.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4000 non-null   int64 
 1   datetime    4000 non-null   object
 2   title       4000 non-null   object
 3   content     4000 non-null   object
 4   category    4000 non-null   object
 5   url         4000 non-null   object
dtypes: int64(1), object(5)
memory usage: 187.6+ KB


In [ ]:
mv_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/mv.csv")
mv_dataset = mv_dataset.rename(columns = {"time": "date"})
mv_dataset["date"] = mv_dataset["date"].str.strip()
mv_dataset = mv_dataset.drop_duplicates()
mv_dataset = mv_dataset[mv_dataset["article"] != "None"]
mv_dataset["publisher"] = "martinvrijland"
mv_dataset["trust"] = "untrusted"
mv_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/mv.csv", index=False)

In [ ]:
nu_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/nu.csv")
nu_dataset = nu_dataset.drop(nu_dataset.columns[0], axis=1)
nu_dataset = nu_dataset[nu_dataset["article"] != "None"]
nu_dataset = nu_dataset.drop_duplicates()
nu_dataset["publisher"] = "nu"
nu_dataset["trust"] = "trusted"
nu_dataset.info()
nu_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/nu.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2124 entries, 0 to 2123
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        2124 non-null   object
 1   title      2124 non-null   object
 2   article    2124 non-null   object
 3   date       2124 non-null   object
 4   publisher  2124 non-null   object
 5   trust      2124 non-null   object
dtypes: object(6)
memory usage: 116.2+ KB


In [ ]:
par_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/parool.csv")
par_dataset = par_dataset.drop(["id", "mainSection"], axis=1)
par_dataset = par_dataset.rename(columns={"text": "article", "published_at": "date"})
par_dataset["date"] = pd.to_datetime(par_dataset["date"])
par_dataset = par_dataset[par_dataset["url"].notna()]
par_dataset = par_dataset[par_dataset["article"].notna()]
par_dataset = par_dataset[["url","title","article","date","publisher"]]
par_dataset["trust"] = "trusted"
par_dataset.info()
par_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/parool.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2435 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        2435 non-null   object        
 1   title      2435 non-null   object        
 2   article    2435 non-null   object        
 3   date       2435 non-null   datetime64[ns]
 4   publisher  2435 non-null   object        
 5   trust      2435 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 133.2+ KB


In [ ]:
reg_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/regional.csv")
reg_dataset = reg_dataset.drop(["id", "mainSection"], axis=1)
reg_dataset = reg_dataset.rename(columns={"text": "article", "published_at": "date"})
reg_dataset["date"] = pd.to_datetime(reg_dataset["date"])
reg_dataset = reg_dataset[["url","title","article","date","publisher"]]
reg_dataset["trust"] = "trusted"
reg_dataset.info()
reg_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/regional.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        2500 non-null   object        
 1   title      2500 non-null   object        
 2   article    2500 non-null   object        
 3   date       2500 non-null   datetime64[ns]
 4   publisher  2500 non-null   object        
 5   trust      2500 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 117.3+ KB


In [ ]:
tel_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/tel.csv")
tel_dataset = tel_dataset[tel_dataset["article"].notna()]
tel_dataset["date"] = tel_dataset["date"].str.extract(r"([0-9]+ mrt. 2021)")
tel_dataset = tel_dataset[tel_dataset["date"].notna()]
tel_dataset = tel_dataset.drop_duplicates()
tel_dataset["publisher"] = "telegraaf"
tel_dataset["trust"] = "trusted"
tel_dataset.info()
tel_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/telegraaf.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4874 entries, 0 to 5248
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        4874 non-null   object
 1   title      4874 non-null   object
 2   article    4874 non-null   object
 3   date       4874 non-null   object
 4   publisher  4874 non-null   object
 5   trust      4874 non-null   object
dtypes: object(6)
memory usage: 266.5+ KB


In [ ]:
trw_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/trouw.csv")
trw_dataset = trw_dataset.drop(["id", "mainSection"], axis=1)
trw_dataset = trw_dataset.rename(columns={"text": "article", "published_at": "date"})
trw_dataset["date"] = pd.to_datetime(trw_dataset["date"])
trw_dataset = trw_dataset[trw_dataset["url"].notna()]
trw_dataset = trw_dataset[trw_dataset["article"].notna()]
trw_dataset = trw_dataset[["url","title","article","date","publisher"]]
trw_dataset["trust"] = "trusted"
trw_dataset.info()
trw_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/trouw.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3226 entries, 0 to 4998
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        3226 non-null   object        
 1   title      3226 non-null   object        
 2   article    3226 non-null   object        
 3   date       3226 non-null   datetime64[ns]
 4   publisher  3226 non-null   object        
 5   trust      3226 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 176.4+ KB


In [ ]:
trans_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/transweb.csv")
trans_dataset = trans_dataset[trans_dataset["article"].notna()]
trans_dataset = trans_dataset.drop_duplicates()
trans_dataset["publisher"] = "transweb"
trans_dataset["trust"] = "untrusted"
trans_dataset.info()
trans_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/transweb.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 478 entries, 0 to 479
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        478 non-null    object
 1   title      478 non-null    object
 2   article    478 non-null    object
 3   date       478 non-null    object
 4   publisher  478 non-null    object
 5   trust      478 non-null    object
dtypes: object(6)
memory usage: 26.1+ KB


In [ ]:
vlk_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/Datasets/volkskrant.csv")
vlk_dataset = vlk_dataset.drop(["id", "mainSection"], axis=1)
vlk_dataset = vlk_dataset.rename(columns={"text": "article", "published_at": "date"})
vlk_dataset["date"] = pd.to_datetime(vlk_dataset["date"])
vlk_dataset = vlk_dataset[vlk_dataset["article"].notna()]
vlk_dataset = vlk_dataset[["url","title","article","date","publisher"]]
vlk_dataset["trust"] = "trusted"
vlk_dataset.info()
vlk_dataset.to_csv("/content/drive/MyDrive/Master Thesis/Cleaned/volkskrant.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2498 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   url        2498 non-null   object        
 1   title      2498 non-null   object        
 2   article    2498 non-null   object        
 3   date       2498 non-null   datetime64[ns]
 4   publisher  2498 non-null   object        
 5   trust      2498 non-null   object        
dtypes: datetime64[ns](1), object(5)
memory usage: 136.6+ KB


In [ ]:
#Shuffle datasets that haven't been yet
sources = ["dlm","mv","niburu","nieuwspaal","ninefornews","nu","speld",
            "telegraaf","transweb"]

for s in sources:
    dataset = pd.read_csv(F"/content/drive/MyDrive/Master Thesis/Cleaned/{s}.csv")
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    dataset.to_csv(F"/content/drive/MyDrive/Master Thesis/Cleaned/{s}.csv", index=False)

In [ ]:
#Shuffle train, dev and test
datasets = ["train","dev","test"]

for d in datasets:
    dataset = pd.read_csv(F"/content/drive/MyDrive/Master Thesis/{d}.csv")
    dataset = dataset.sample(frac=1).reset_index(drop=True)
    dataset.to_csv(F"/content/drive/MyDrive/Master Thesis/{d}.csv", index=False)

In [2]:
train = pd.read_csv("/content/drive/MyDrive/Master Thesis/train.csv")
no_article = train[(train["article"].isna()) | (train["article"] == "None")]
print(no_article["publisher"].value_counts())
train = train[train["article"].notna()]
train = train[train["article"] != "None"]
train["article"] = train["article"].str.strip()
print(train["publisher"].value_counts())
train.info()
#train.to_csv("/content/drive/MyDrive/Master Thesis/train.csv", index=False)

Series([], Name: publisher, dtype: int64)
niburu               3500
speld                3000
ninefornews          2500
nos                  2000
nu                   2000
trouw                2000
vk                   2000
eindhovensdagblad    1485
martinvrijland       1000
pzc                   139
destentor             117
degelderlander        110
brabantsdagblad        89
destem                 39
tubantia               21
Name: publisher, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        20000 non-null  object
 1   title      20000 non-null  object
 2   article    20000 non-null  object
 3   date       20000 non-null  object
 4   publisher  20000 non-null  object
 5   trust      20000 non-null  object
dtypes: object(6)
memory usage: 1.1+ MB


In [3]:
dev = pd.read_csv("/content/drive/MyDrive/Master Thesis/dev.csv")
dev = dev[dev["article"].notna()]
dev = dev[dev["article"] != "None"]
dev["article"] = dev["article"].str.strip()
print(dev["publisher"].value_counts())
print(dev.info())
#dev.to_csv("/content/drive/MyDrive/Master Thesis/dev.csv", index=False)

nos            314
niburu         314
vk             312
ad             312
dlmplus        312
parool         312
ninefornews    312
nieuwspaal     312
Name: publisher, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        2500 non-null   object
 1   title      2500 non-null   object
 2   article    2500 non-null   object
 3   date       2500 non-null   object
 4   publisher  2500 non-null   object
 5   trust      2500 non-null   object
dtypes: object(6)
memory usage: 136.7+ KB
None


In [11]:
test = pd.read_csv("/content/drive/MyDrive/Master Thesis/test.csv")
test = test[test["article"].notna()]
test= test[test["article"] != "None"]
test["article"] = test["article"].str.strip()
print(test["publisher"].value_counts())
print(test.info())
#test.to_csv("/content/drive/MyDrive/Master Thesis/test.csv", index=False)

telegraaf     418
transweb      417
nieuwspaal    417
ad            416
dlmplus       416
parool        416
Name: publisher, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        2500 non-null   object
 1   title      2500 non-null   object
 2   article    2500 non-null   object
 3   date       2500 non-null   object
 4   publisher  2500 non-null   object
 5   trust      2500 non-null   object
dtypes: object(6)
memory usage: 136.7+ KB
None
telegraaf     418
transweb      417
nieuwspaal    417
ad            416
dlmplus       416
parool        416
Name: publisher, dtype: int64


In [12]:
# Merge train, dev and test set into one dataset
full = pd.concat([train,dev,test])

full.info()

full.to_csv("/content/drive/MyDrive/Master Thesis/full_dataset.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25000 entries, 0 to 2499
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        25000 non-null  object
 1   title      25000 non-null  object
 2   article    25000 non-null  object
 3   date       25000 non-null  object
 4   publisher  25000 non-null  object
 5   trust      25000 non-null  object
dtypes: object(6)
memory usage: 1.3+ MB


In [13]:
#Take 500 random instances from the full dataset for annotation

full_dataset = pd.read_csv("/content/drive/MyDrive/Master Thesis/full_dataset.csv")
full_data_sample = full_dataset.sample(500, replace=False, random_state=1)
full_data_sample.to_csv("/content/drive/MyDrive/Master Thesis/annotation_set.csv")

In [ ]:
frames = [train,dev,test]
full_dataset = pd.concat(frames)
print(full_dataset["publisher"].value_counts())

date = full_dataset["date"][full_dataset["publisher"] == "transweb"].value_counts().sort_index()
print(date)